In [2]:
import sklearn
import numpy as np
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import re
import pickle
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

In [3]:
conn = sqlite3.connect('../db.sqlite3')

In [4]:
df_raw = pd.read_sql_query("SELECT * FROM scraper_auto WHERE bron='marktplaats'", conn)

In [5]:
target_names = ["bouwjaar", "kilometer_stand", "vermogen", "prijs", "is_handgeschakeld", "is_benzine", "upload_datum", "apk", "titel"]
df = df_raw[target_names]
df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine
count,520.000000,518.000000,520.000000,508.000000,518.00000,520.000000
mean,2007.819231,178712.822394,80.794231,4575.324803,0.96139,0.740385
std,1.762928,57989.211249,13.868011,44262.476128,0.19285,0.438846
min,2005.000000,15570.000000,51.000000,1.000000,0.00000,0.000000
25%,2006.000000,138166.250000,76.500000,1950.000000,1.00000,0.000000
50%,2007.000000,177226.500000,78.000000,2550.000000,1.00000,1.000000
75%,2009.000000,211835.750000,84.000000,3250.000000,1.00000,1.000000
max,2014.000000,368884.000000,155.000000,999999.000000,1.00000,1.000000


In [6]:
MAX_PRICE = 30000
MIN_PRICE = 1000
df = df[(df['prijs'] > MIN_PRICE) & (df['prijs'] < MAX_PRICE)]

MIN_BOUWJAAR = 2000
df = df[df['bouwjaar'] > MIN_BOUWJAAR]

MAX_VERMOGEN = 500
df = df[df['vermogen'] < MAX_VERMOGEN]

df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine
count,481.000000,479.000000,481.000000,481.000000,479.000000,481.000000
mean,2007.904366,174594.037578,80.392931,2716.505198,0.960334,0.750520
std,1.779279,54573.111684,13.231806,876.023854,0.195377,0.433163
min,2005.000000,15570.000000,51.000000,1150.000000,0.000000,0.000000
25%,2006.000000,136983.500000,75.000000,1999.000000,1.000000,1.000000
50%,2007.000000,176368.000000,78.000000,2650.000000,1.000000,1.000000
75%,2009.000000,207064.500000,84.000000,3250.000000,1.000000,1.000000
max,2014.000000,368884.000000,155.000000,6950.000000,1.000000,1.000000


In [7]:
df['upload_datum'] = pd.to_datetime(df['upload_datum'])
df['apk'] = pd.to_datetime(df['apk'])

average_upload_datum = df['upload_datum'].mean().toordinal()
average_apk =df['apk'].mean().toordinal()

def upload_datum_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_upload_datum)
    else:
        return abs(datetime.now().toordinal() - date.toordinal())
    
def apk_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_apk)
    else:  
        return abs(datetime.now().toordinal() - date.toordinal())

df['upload_datum'] = df['upload_datum'].apply(upload_datum_to_ordinal).astype(int)
df['apk'] = df['apk'].apply(apk_to_ordinal).astype(int)

def parse_titel(titel):
    cleaned = re.sub('[^a-zA-Z]+', ' ', titel.lower())
    stripped = line = re.sub(r'\b\w{1,3}\b', '', cleaned)
    
    return stripped

df['titel'] = df['titel'].apply(parse_titel)

In [8]:
df = df.dropna()
#df.isna().sum()

In [9]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
df.to_pickle("./data.pkl")